## **Notebook 03: LangGraph Agent Prototype**

**Purpose**
Build the core agentic workflow using LangGraph. This notebook establishes the multi-agent system that will power Radar's paper analysis pipeline.

**What We'll Do**

| Step | Task | Output |
|------|------|--------|
| 1 | **Setup LangGraph** | Import libraries and configure LLM client |
| 2 | **Define Agent State** | Create state schema for data flow between agents |
| 3 | **Build Paper Analyzer** | Agent that extracts key technical insights from papers |
| 4 | **Build Simplifier** | Agent that generates accessible explanations |
| 5 | **Build Industry Matcher** | Agent that maps research to use cases |
| 6 | **Orchestrate Workflow** | Connect agents in LangGraph state machine |
| 7 | **Test Pipeline** | Run complete workflow on sample papers |

**Key Questions to Answer**
- How do I structure state in LangGraph for multi-agent workflows?
- What prompts effectively extract technical insights from papers?
- How do I chain agents while maintaining context?
- Can the pipeline handle multiple papers efficiently?

**Expected Outcomes**
- Working LangGraph state machine with 3+ agents
- Tested prompts for paper analysis and simplification
- Complete pipeline: Paper text -> Technical analysis -> Simple summary -> Industry applications
- Performance baseline for optimization

**Architecture**
```
Paper Text (from Notebook 02)
    |
    v
Paper Analyzer Agent
    |
    v
Simplifier Agent  
    |
    v
Industry Matcher Agent
    |
    v
Final Output (structured JSON)
```

---



In [5]:
# Imports and Setup

"""
Import LangGraph, LangChain, and Anthropic libraries.
Configure Claude API client for agent operations.
"""

# Core libraries
import json
import os
from typing import TypedDict, Annotated
from datetime import datetime

# LangGraph and LangChain
from langgraph.graph import StateGraph, END
from langchain_anthropic import ChatAnthropic

# Load environment variables
from dotenv import load_dotenv
load_dotenv()



True

In [6]:
# Verify API key
api_key = os.getenv('ANTHROPIC_API_KEY')
if not api_key:
    raise ValueError("ANTHROPIC_API_KEY not found in environment variables")

# Initialize Claude client
llm = ChatAnthropic(
    model="claude-sonnet-4-20250514",
    temperature=0.7,
    max_tokens=4096
)

print("Setup complete")
print(f"LLM model: claude-sonnet-4-20250514")
print(f"API key loaded: {api_key[:8]}...")

Setup complete
LLM model: claude-sonnet-4-20250514
API key loaded: sk-ant-a...


In [7]:
# Cell 3: Load Sample Papers from Notebook 02

"""
Load the processed papers to use as input for our agents.
"""

import pandas as pd

# Load processed data
processed_path = '../data/processed/processed_papers_sample.json'

with open(processed_path, 'r', encoding='utf-8') as f:
    processed_papers = json.load(f)

print("LOADED PROCESSED PAPERS")
print("=" * 80)
print(f"Papers available: {len(processed_papers)}")

for i, paper in enumerate(processed_papers[:3], 1):
    print(f"\n{i}. {paper['title'][:60]}...")
    print(f"   Pages: {paper['page_count']} | Chars: {paper['char_count']:,}")
    print(f"   Sections: {paper['section_count']}")

print("\nReady to process with agents")

LOADED PROCESSED PAPERS
Papers available: 5

1. Manifold limit for the training of shallow graph convolution...
   Pages: 44 | Chars: 117,139
   Sections: 4

2. AdaFuse: Adaptive Ensemble Decoding with Test-Time Scaling f...
   Pages: 13 | Chars: 52,133
   Sections: 5

3. Chaining the Evidence: Robust Reinforcement Learning for Dee...
   Pages: 21 | Chars: 74,187
   Sections: 5

Ready to process with agents


In [8]:
# Cell 4: Define LangGraph State

"""
Define the state structure that will flow between agents.
Each agent reads from and writes to this shared state.
"""

class AgentState(TypedDict):
    """
    State schema for our agent workflow.
    Each agent can read and modify these fields.
    """
    # Input
    paper_title: str
    paper_text: str
    paper_sections: dict
    
    # Paper Analyzer outputs
    technical_summary: str
    key_methods: str
    main_results: str
    limitations: str
    
    # Simplifier outputs
    executive_summary: str
    key_innovation: str
    accessible_explanation: str
    
    # Metadata
    processing_stage: str
    errors: list

print("State schema defined: AgentState")
print("\nState fields:")
print("  Input: paper_title, paper_text, paper_sections")
print("  Analyzer: technical_summary, key_methods, main_results, limitations")
print("  Simplifier: executive_summary, key_innovation, accessible_explanation")
print("  Meta: processing_stage, errors")

State schema defined: AgentState

State fields:
  Input: paper_title, paper_text, paper_sections
  Analyzer: technical_summary, key_methods, main_results, limitations
  Simplifier: executive_summary, key_innovation, accessible_explanation
  Meta: processing_stage, errors
